In [1]:
import pandas as pd
import networkx as nx
import itertools

In [2]:
df = pd.read_json("rawfilteredbydate.json")

In [3]:
df.head()

,caption,date
0,The Perlman Music Program Class of 2014,2014-08-22
1,Toby Perlman addresses the crowd,2014-08-22
2,Itzhak Perlman and Catherine Cho perform with...,2014-08-22
3,"Julian Beck, Lyn Barlow, Ted Ayala, Elinor He...",2014-08-22
4,Toby Perlman and guests,2014-08-22


In [4]:
df["caption"] = df['caption'].apply(lambda x: x.replace("\n",""))

In [5]:
df = df[df['caption'].apply(lambda x: len(x)<=250)]

In [159]:
restr = ["The", "Dr", "Verdana", "Helvetica" ,"Chairman","Director",
         "Guests","President","Hospital","Special Surgery","Executive Director"
         ,"New York","Museum","Photographs","Children","The Society","Dinner",
        "Board Member","Castle","President and CEO","Directors","Clockwise","Doubles","BFAnyc","Standing",
        "Susan Magazine","Trustees","Chicago","Yugoslavia","Steering Committee",
        "I Have A Dream"]

In [175]:
prereplace = ["friend","friends"]

In [176]:
def replaceWord(s):
    for i in range(len(prereplace)):
        s = s.replace(prereplace[i],"")
    return s

In [177]:
captions = map(lambda x:replaceWord(x),df.caption)

In [178]:
proper_noun_regex = r'([A-Z][A-Za-z-]*(,?\s+(\b(and|von|de|van der|van|der)\b))?(\s[A-Z][A-Za-z\'-]*)*)'

In [179]:
import re

In [180]:
G=nx.MultiGraph()
for caption in captions:
    name = []
    caption_temp  = re.findall(proper_noun_regex,caption)
    for j in range(len(caption_temp)):
        name_temp = caption_temp[j]
        if len(name_temp[0])>5 and len(name_temp[0].split(" ")) > 1 and len(name_temp[0].split(" ")) <=4:
            
            if name_temp[0] == "PatrickMcMullan":
                name.append("Patrick McMullan")
            elif name_temp[0] == "AnnieWatt":
                name.append("Annie Watt")
            elif "Mayor" in name_temp[0]:
                temp = name_temp[0].replace("Mayor ","")
                name.append(temp)
            elif len(name_temp[0].split(" and ")) == 2:
                name1 = name_temp[0].split(" and ")[0]
                name2 = name_temp[0].split(" and ")[1]
                if len(name1.split(" ")) == 1 and len(name2.split(" ")) == 2:
                    name.append("")
                    name.append()

                    
            else:
                name.append(name_temp[0])
        for word in restr:
            if word in name:
                name.remove(word)
    if len(name) > 2:
        _=[G.add_edge(c[0],c[1],weight=1) for c in itertools.combinations(name,2)]
    elif len(name) > 1:
        G.add_edge(name[0],name[1],weight=1)        

In [181]:
Gg = nx.Graph()
for u,v,data in G.edges_iter(data=True):
    if Gg.has_edge(u,v):
        Gg[u][v]['weight'] += 1
    else:
        Gg.add_edge(u, v, weight=1)

## degrees

In [182]:
degrees=pd.DataFrame(G.degree().values(),index=G.degree().keys()).sort(0,ascending=False)

In [183]:
degrees = degrees.reset_index()

In [184]:
degrees[180:220]

,index,0
180,Michael Cominotto,100
181,Princess Yasmin Aga Khan,99
182,Jackie Harris Hochberg,99
183,Dorothy Lichtenstein,99
184,Julia Koch,99
185,Melanie Holland,98
186,Mary Alice Stephenson,98
187,Veronica Kelly,98
188,Kirk Henckels,98
189,Olivia Chantecaille,97


In [185]:
degrees.columns = ["name","number"]

In [186]:
[i for i in degrees.name if " and " in i]

[u'Alan and Susan Patricof',
 u'Chairman and Gala Co-chair',
 u'John and Margo Catsimatidis',
 u'Chairman and CEO',
 u'Shamon, and Diamon Hill',
 u'Cynthia and Dr',
 u'Justice and Human Rights',
 u'Michael and Eleanora Kennedy',
 u'Barbara and Donald Tober',
 u'Kelly, and Dr',
 u'Giuseppe and Antonietta Bono',
 u'Absurdistan, and Annie Eggers',
 u'Director and Chief Curator',
 u'Sex, and Drugs',
 u'Katharine and William Rayner',
 u'Gianni and Daniela Onorato',
 u'Yugoslavia, and Cornelia Guest',
 u'Heuvel, and Sally Minard',
 u'Caputo, and Kate Hufft',
 u'Alex and Kate Donner',
 u'Donald and Barbara Tober',
 u'Tatiana and Campion Platt',
 u'Ambassador and Mrs',
 u'Toth, and Drew Perry',
 u'Castle, and Ingrid Connolly',
 u'Grace and Chris Meigher',
 u'Dana and Evan Schiff',
 u'Andrew and Zibby Right',
 u'John and Aileen Crowley',
 u'Valentin and Yaz Hernandez',
 u'Klara and Larry Silverstein',
 u'MD, and Daniel Davis',
 u'Elizabeth, and Giulian Mota',
 u'Hailey and Mama',
 u'Patricia an

In [187]:
degreeres = map(lambda x,y: (x,y),degrees.name[0:100], degrees.number[0:100])

In [188]:
import pickle

In [189]:
with open('degreeres.pickle', 'wb') as f:
    pickle.dump(degreeres, f)

In [14]:
pr=nx.pagerank(Gg)
pranalysis=pd.DataFrame(pr.values(),index=pr.keys()).sort(0,ascending=False)